In [ ]:
# default_exp utils

# Utils

> contains various util functions and classes 

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import re

import pandas as pd
import numpy as np
from random import randrange

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter

from fastai.torch_basics import *
from fastai.basics import *
from fastai.metrics import accuracy
from fastai.learner import *
from fastai.callback.all import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler
%matplotlib inline

# Functions

In [ ]:
#export
def f1score(truth, classified):
    'Calculates F1 score given a list of the truth and classified values'
    tp = len(set(truth).intersection(set(classified)))
    t = len(truth)
    p = len(classified)
    
    if (p == 0):
         precision = 1
    else:
        precision = tp/p
        
    if (t == 0):
        recall = 1
    else:
        recall = tp/t
    
    if (precision == 0 and recall == 0):
        f1 = 0
    else:
        f1 = 2*precision*recall/(precision+recall)
    return f1, tp, t, p

In [ ]:
#export
def df_preproc (df):
    'Prepocesses the df to be ready for Anomaly Detection it will add start/end events to every trace'
    df['event_id']= df.index
    df.index = df['trace_id']
    df = df[["event_id", "activity", "trace_id"]]
    
    for i in df['trace_id'].unique():
        df_cop = df.loc[i]
        df.loc[i, 'event_id']  = np.arange(1, len(df_cop)+1)
        
    df.reset_index(drop=True, inplace=True)
    
    trace_ends = list(df.loc[df['event_id']==1].index)[1:]
    trace_ends.append(len(df))
    new = np.insert(np.array(df),trace_ends, [-1, 'end', 0], axis=0)
    df = pd.DataFrame(data=new, columns=["event_id", "activity", "trace_id"])
    
    trace_starts = list(df.loc[df['event_id']==1].index)
    new = np.insert(np.array(df),trace_starts, [0, 'start', 0], axis=0)
    
    trace_starts = np.where(new[:,0]==0)[0]
    trace_ends = np.where(new[:,0]==-1)[0]
    new[trace_starts,2] = new[np.array(trace_starts)+1,2]
    new[trace_ends,2] = new[np.array(trace_starts)+1,2]
    new[trace_ends,0] = new[trace_ends-1,0]+1
    
    
    df = pd.DataFrame(data=new, columns=["event_id", "activity", "trace_id"])
    df.index = df['trace_id']

    return df

In [ ]:
#export
def load_data (data='PDC2016'):
    'loads eventlogs from different sources, returns event_df with start/end event, test_df and truth_df'
    
    #---------------PDC2016|2017--------------------------------------------------
    
    if (data == 'PDC2016' or data == 'PDC2017'):
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train'
        files = [filename for filename in os.listdir(directory) if filename.split('.')[-1] =='csv']
        files.sort()
        pick = randrange(len(files)) #picks random eventlog
        df = pd.read_csv('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train/' + files[pick], sep=',')
        df = df.rename(columns={"act_name": "activity", "case_id": "trace_id"})
        event_df = df_preproc(df)
        
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test'
        files = [filename for filename in os.listdir(directory)]
        files.sort()
        log = xes_importer.apply('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test/' + files[pick])
        df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
        df = df.rename(columns={"concept:name": "activity", "case:concept:name": "trace_id"})
        df = df[['activity', 'trace_id']]
        test_df = df_preproc(df)
        
        truth = [log[i].attributes['pdc:isPos'] for i in range(len(log))]
        d = {'case': range(1, len(log)+1), 'normal': truth}
        truth_df = pd.DataFrame (data=d)
    
    #---------------PDC2019--------------------------------------------------
    
    if (data == 'PDC2019'):
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train'
        files = [filename for filename in os.listdir(directory) if filename.split('.')[-1] =='csv']
        files.sort()
        pick = randrange(len(files)) #picks random eventlog
        df = pd.read_csv('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train/' + files[pick], sep=',')
        df = df.rename(columns={"event": "activity", "case": "trace_id"})
        df = df[['activity', 'trace_id']]
        event_df = df_preproc(df)
        
    
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test'
        files = [filename for filename in os.listdir(directory)]
        files.sort()
        log = xes_importer.apply('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test/' + files[pick])
        df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
        df = df.rename(columns={"concept:name": "activity", "case:concept:name": "trace_id"})
        df = df[['activity', 'trace_id']]
        test_df = df_preproc(df)
        
        truth = [log[i].attributes['pdc:isPos'] for i in range(len(log))]
        d = {'case': range(1, len(log)+1), 'normal': truth}
        truth_df = pd.DataFrame (data=d)
        
        
    #---------------PDC2020--------------------------------------------------
    if (data == 'PDC2020'):
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train'
        files = [filename for filename in os.listdir(directory) if filename.split('.')[-1] =='xes']
        files.sort()
        pick = randrange(len(files)) #picks random eventlog
        
        log = xes_importer.apply('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test/' + files[pick])
        df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
        df = df.rename(columns={"concept:name": "activity", "case:concept:name": "trace_id"})
        df['trace_id']= df['trace_id'].apply(lambda x: int(x.split()[1]))
        df = df[['activity', 'trace_id']]
        event_df = df_preproc(df)
        
        test_df = event_df
        
        truth = [log[i].attributes['pdc:isPos'] for i in range(len(log))]
        d = {'case': range(1, len(log)+1), 'normal': truth}
        truth_df = pd.DataFrame (data=d)
    
    return event_df, test_df, truth_df

In [ ]:
#export
class TestModel(nn.Module):
    def __init__(self, pp_data ,is_cuda=False,vocab_col='activity'):
        super().__init__()
        vocab_size=len(pp_data.procs.categorify[vocab_col])
        self.vocab_index={s:i for i,s in enumerate(pp_data.cat_names[0])}[vocab_col]
        n_fac, n_hidden=round(sqrt(vocab_size))+1, round(sqrt(vocab_size)*2)
        self.n_hidden=n_hidden
        self.is_cuda=is_cuda
        self.e = nn.Embedding(vocab_size,n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_bottleneck = nn.Linear(n_hidden, 2)
        self.l_out = nn.Linear(2, vocab_size)
        
    def forward(self, xb):
        cs=xb.permute(1,2,0)[self.vocab_index]
        bs = len(cs[0])
        h = torch.zeros((bs,self.n_hidden))
        if self.is_cuda: h=h.cuda()
        for c in cs:
            inp = torch.relu(self.l_in(self.e(c)))
            h = torch.tanh(self.l_hidden(h+inp))
        h = self.l_bottleneck(h)
        return F.log_softmax(self.l_out(h),dim=0)

In [ ]:
#export
class transform(ItemTransform):
    def encodes(self,e):
        (ecats,econts,tcats,tconts),(ycats,yconts)=e
        return (ecats),ycats

In [ ]:
#export
def _shift_columns (a,ws=3): return np.dstack(list(reversed([np.roll(a,i) for i in range(0,ws)])))[0]
def windows_fast(df,event_ids,ws=5,pad=None):
    max_trace_len=int(event_ids.max())+1
    trace_start = np.where(event_ids == 0)[0]
    trace_len=[trace_start[i]-trace_start[i-1] for i in range(1,len(trace_start))]+[len(df)-trace_start[-1]]
    idx=[range(trace_start[i]+(i+1)
               *(ws-1),trace_start[i]+trace_len[i]+(i+1)*(ws-1)) for i in range(len(trace_start))]
    idx=np.array([y for x in idx for y in x])
    trace_start = np.repeat(trace_start, ws-1)
    tmp=np.stack([_shift_columns(np.insert(np.array(df[i]), trace_start, 0, axis=0),ws=ws) for i in list(df)]) 
    tmp=np.rollaxis(tmp,1) 
    res=tmp[idx]
    if pad: res=np.pad(res,((0,0),(0,0),(pad-ws,0))) 
    
    return res[:-1],np.array(range(1,len(res)))

In [ ]:
data = 'PDC2020'

In [ ]:
directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train'
files = [filename for filename in os.listdir(directory) if filename.split('.')[-1] =='xes']
files.sort()
pick = randrange(len(files)) #picks random eventlog

log = xes_importer.apply('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test/' + files[pick])
df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
df = df.rename(columns={"concept:name": "activity", "case:concept:name": "trace_id"})
df['trace_id']= df['trace_id'].apply(lambda x: int(x.split()[1]))
df = df[['activity', 'trace_id']]
event_df = df_preproc(df)

parsing log, completed traces ::   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
event_df

,event_id,activity,trace_id
trace_id,,,
1,0,start,1
1,1,t11,1
1,2,t26,1
1,3,t35,1
1,4,t62,1
...,...,...,...
1000,31,t81,1000
1000,32,t75,1000
1000,33,t82,1000


In [ ]:
event_df, test_df, df_truth = load_data(data='PDC2017')

parsing log, completed traces ::   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
df_truth

,case,normal
0,1,False
1,2,False
2,3,True
3,4,True
4,5,True
5,6,False
6,7,True
7,8,False
8,9,False
9,10,True


In [ ]:
df

,activity,trace_id,event_id
trace_id,,,
1,t11,1,0
1,t21,1,1
1,t32,1,2
1,t26,1,3
1,t44,1,4
...,...,...,...
1000,t76,1000,34486
1000,t82,1000,34487
1000,t71,1000,34488
